# Import thư viện cần thiết

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # seaborn là thư viện được xây trên matplotlib, giúp việc visualization đỡ khổ hơn
import pandas as pd
import numpy as np
import re
import unicodedata

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPRegressor
from sklearn.impute import KNNImputer
# You can also import other things ...
# YOUR CODE HERE (OPTION)

## Một số xử lý trước khi chia tập train, test

#### Đọc file dữ liệu và đổi tên cột

In [2]:
df = pd.read_csv("raw_data3.csv",sep='\t')

df.columns = ["link", "name", "type", "area", "room_types", "price of the week", "location","utilities", "Weekend prices"]
df.head()

,link,name,type,area,room_types,price of the week,location,utilities,Weekend prices
0,https://www.luxstay.com/vi/rooms/13967,JOLIE HOUSE - BIỆT THỰ HƯỚNG RA VƯỜN - CÁCH TR...,Biệt Thự,100 m2,Nguyên căn · 5 phòng tắm · 6 giường · 4 phòng ...,"2,887,500₫ /đêm","Đà Lạt, Lâm Đồng, Vietnam","Tiện ích gia đình,Phù hợp với trẻ nhỏ,Đệm bổ s...","3,280,200₫"
1,https://www.luxstay.com/vi/rooms/33714,Gold&Ruby House- Phòng Quadruple,Căn hộ dịch vụ,20 m2,Phòng riêng · 1 phòng tắm · 2 giường · 1 phòng...,"600,000₫ /đêm","Đà Lạt, Lâm Đồng, Vietnam","Tiện ích gia đình,Phù hợp với trẻ nhỏ,Không hú...",NaN
2,https://www.luxstay.com/vi/rooms/15456,Maika Condotel - Triple Bedroom,Căn hộ chung cư,110 m2,Nguyên căn · 2 phòng tắm · 3 giường · 3 phòng ...,"2,750,000₫ /đêm","Đà Lạt, Lâm Đồng, Vietnam","Tiện ích gia đình,Phù hợp với trẻ nhỏ,Tiện ích...","2,950,000₫"
3,https://www.luxstay.com/vi/rooms/18878,Vườn đom đóm Đà Lạt- Phòng đơn hướng vườn rộng...,Nhà riêng,12 m2,Phòng riêng · 3 phòng tắm · 1 giường · 1 phòng...,"370,000₫ /đêm","Đà Lạt, Lâm Đồng, Vietnam","Tiện ích gia đình,Không hút thuốc,Tiện ích bếp...","470,000₫"
4,https://www.luxstay.com/vi/rooms/15377,Rebeka's Home 1 - Góc vườn nhỏ tĩnh lặng trước...,Nhà riêng,15 m2,Phòng riêng · 1 phòng tắm · 1 giường · 1 phòng...,"350,000₫ /đêm","Đà Lạt, Lâm Đồng, Vietnam","Tiện ích gia đình,Phù hợp với trẻ nhỏ,Tiện ích...",NaN


#### Do lúc lấy thông tin 1 địa điểm có 2 giá là giá cuối tuần và giá trong tuần. Nghĩa là 1 dòng có đến 2 output, vì vậy ta tiến hành xử lý đưa về 1 output và output này phải là số.

In [3]:
# Bỏ các dòng có giá bằng nan
df=df[ pd.isna(df['price of the week'])!=1]

# Do thời gian lấy dữ liệu cuối tuần và trong tuần ở 2 thời điểm khác nhau nên một số dữ liệu không còn 
# vì vậy ta sẽ lấy giá cuối tuần bằng giá trong tuần đối với các giá trị null
df['Weekend prices']=df.apply(lambda x:x['price of the week'] if pd.isna(x['Weekend prices']) else x['Weekend prices'],axis=1)

# Tiến hành gộp 2 cột về 1 cột
df['prices'] = df[['price of the week', 'Weekend prices']].apply(lambda x:[x['price of the week'],x['Weekend prices']], axis=1)
df=df.drop("price of the week",axis=1)
df=df.drop("Weekend prices",axis=1)
df.head()

# Thêm 2 cột thể hiện chỗ ở được thuê trong tuần hay cuối tuần
df['inweek']=1
df['weekend']=0

# Tạo 1 dataframe mới giống với df nhưng khác cột trong tuần và cuối tuần
temp=df.copy()
df['inweek']=0
df['weekend']=1

# Tiến hành nối 2 dataframe này lại với nhau
last_df=pd.concat([df, temp],ignore_index=True)

# Thêm một cột hiển thị giá cuối cùng theo trong tuần hay cuối tuần
last_df['last_prices']=np.nan
last_df['last_prices']=last_df.apply(lambda x: x['prices'][0] if (x['inweek']==1 or x['prices'][1]==np.nan) else x['prices'][1]  ,axis=1)
last_df=last_df.drop('prices',axis=1)

# Chuyển chuỗi ở cột last_prices thành số
last_df['last_prices']=last_df['last_prices'].str.split(" ").apply(lambda x: x[0][:-1].replace(",",""))

# Bỏ đi các giá trị không thể chuyển thành số 
last_df=last_df[last_df['last_prices'].str.isnumeric()]

# Chuyển kiểu dữ liệu về số
last_df['last_prices'] = pd.to_numeric(last_df['last_prices'])

#### Tiến hành chia file train và file test

In [4]:
# y_sr = last_df["last_prices"] # sr là viết tắt của series
# X_df = last_df.drop("last_prices", axis=1)

# # Tách tập train và tập validation theo tỉ lệ 80%:20%
# train_X_df, test_X_df, train_y_sr, test_y_sr = train_test_split(X_df, y_sr, test_size=0.2, random_state=0)

# # Ghi X_train xuống file csv
# train_X_df.to_csv("X_train.csv",index_label="ID",sep="\t")

# # Ghi y_train xuống file csv
# train_y_sr.to_frame('Price').to_csv("y_train.csv",index_label="ID",sep="\t")

# # Ghi X_test xuống file csv
# test_X_df.to_csv("X_test.csv",index_label="ID",sep="\t")

# # Ghi y_test test xuống file csv
# test_y_sr.to_frame('Price').to_csv("y_test.csv",index_label="ID",sep="\t")

---
# Tiền xử lý

### Bỏ, thêm các cột, chuyển về kiểu dữ liệu phù hợp

In [5]:
# Load X_train
X_df = pd.read_csv('X_train.csv', index_col=0,sep="\t")

# Load y_train
y_sr = np.ravel(pd.read_csv('y_train.csv', index_col=0,sep="\t"))

In [6]:
# Tách tập train và tập validation theo tỉ lệ 75%:25%
train_X_df, val_X_df, train_y_sr, val_y_sr = train_test_split(X_df, y_sr, test_size=0.25, random_state=0)
X_df=train_X_df.copy()

In [7]:
train_X_df.dtypes

link          object
name          object
type          object
area          object
room_types    object
location      object
utilities     object
inweek         int64
weekend        int64
dtype: object

### Xử lý cột link và name

Ta dễ thấy hai cột trên không giúp gì trong quá trình học nên ta sẽ bỏ nó đi

In [8]:
class LinkNameDropper(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        return self
    def transform(self, X_df, y=None):
        # Copy ra 1 dataframe mới để không thay đổi X_df
        X_df_transform=X_df.copy()
        
        # Bỏ cột link
        X_df_transform=X_df_transform.drop("link",axis=1)
        
        # Bỏ cột name
        X_df_transform=X_df_transform.drop("name",axis=1)
        
        return X_df_transform

In [9]:
link_name_dropper =LinkNameDropper()

train_X_df = link_name_dropper.transform(train_X_df)

train_X_df.head()

,type,area,room_types,location,utilities,inweek,weekend
ID,,,,,,,
64,Nhà riêng,9 m2,Phòng riêng · 3 phòng tắm · 1 giường · 1 phòng...,"Đà Lạt, Lâm Đồng, Vietnam","Tiện ích gia đình,Đệm bổ sung,Không hút thuốc,...",0,1
2521,Căn hộ chung cư,78 m2,Nguyên căn · 2 phòng tắm · 2 giường · 2 phòng ...,"Ba Đình, Hà Nội, Vietnam","Tiện ích gia đình,Phù hợp với trẻ nhỏ,Không hú...",0,1
9018,Khác,25 m2,Phòng riêng · 1 phòng tắm · 2 giường · 1 phòng...,"Hoàn Kiếm, Hà Nội, Vietnam","Tiện ích gia đình,Không hút thuốc,Tiện ích bếp...",1,0
11468,Khác,20 m2,Nguyên căn · 3 phòng tắm · 12 giường · 1 phòng...,"Tân Bình, Hồ Chí Minh, Vietnam","Tiện ích gia đình,Không hút thuốc,Tiện ích phò...",1,0
12199,Căn hộ dịch vụ,60 m2,Phòng riêng · 2 phòng tắm · 1 giường · 1 phòng...,"Quận 3, Hồ Chí Minh, Vietnam","Tiện ích gia đình,Không hút thuốc,Tiện ích bếp...",1,0


### Xử lý cột type

In [10]:
# Xem các giá trị trong cột type
train_X_df['type'].value_counts()

Căn hộ chung cư    2528
Căn hộ dịch vụ     2074
Nhà riêng          1949
Căn hộ Studio      1145
Khác                678
Biệt Thự            608
Name: type, dtype: int64

In [11]:
class TypeTransform(BaseEstimator, TransformerMixin):
    def __init__(self, num_top_types=1):
        self.num_top_types= num_top_types
    def fit(self, X_df, y=None):
        type_col = X_df['type']
        self.type_counts_ = type_col.value_counts()
        types = list(self.type_counts_.index)
        self.top_types_ = types[:max(1, min(self.num_top_types, len(types)))]
        return self
    def transform(self, X_df, y=None):
        # Copy ra 1 dataframe mới để không thay đổi X_df
        X_df_transform=X_df.copy()
        
        # Lấy cột type
        type_s=X_df_transform['type']
        
        # Chuyển các giá trị trong cột type về các giá trị trong top_types_.
        # Do trong các giá trị đã có sẵn loại giá trị "Khác" nên 
        # ta sẽ dùng giá trị này để đánh dấu dữ liệu bị thiếu
        type_s=type_s.apply(lambda x: x if x in self.top_types_ else "Khác")
        
        # Thêm cột title vào dataframe
        X_df_transform['type']=type_s
        
        return X_df_transform

In [12]:
type_transform = TypeTransform(num_top_types=3)
type_transform.fit(train_X_df)
print(type_transform.type_counts_)
print()
print(type_transform.top_types_)

train_X_df = type_transform.transform(train_X_df)
train_X_df['type'].value_counts()

Căn hộ chung cư    2528
Căn hộ dịch vụ     2074
Nhà riêng          1949
Căn hộ Studio      1145
Khác                678
Biệt Thự            608
Name: type, dtype: int64

['Căn hộ chung cư', 'Căn hộ dịch vụ', 'Nhà riêng']


Căn hộ chung cư    2528
Khác               2431
Căn hộ dịch vụ     2074
Nhà riêng          1949
Name: type, dtype: int64

### Xử lý cột area

In [13]:
train_X_df.dtypes

type          object
area          object
room_types    object
location      object
utilities     object
inweek         int64
weekend        int64
dtype: object

#### Chuyển kiểu dữ liệu về dạng số

In [14]:
class AreaTransform(BaseEstimator, TransformerMixin):    
    def fit(self, X_df, y=None):
        return self
    def transform(self, X_df, y=None):
        # Copy ra 1 dataframe mới để không thay đổi X_df
        X_df_transform=X_df.copy()
        
        # Lấy cột area
        area=X_df_transform['area']
        
        # Bỏ m2
        area= area.str.replace("m2","")
        X_df_transform['area']=area
        
        
        # Chuyển về thành số
        X_df_transform['area'] = pd.to_numeric(X_df_transform['area'])
        return X_df_transform

In [15]:
area_transform=AreaTransform()
train_X_df=area_transform.transform(train_X_df)
train_X_df.head()

,type,area,room_types,location,utilities,inweek,weekend
ID,,,,,,,
64,Nhà riêng,9.0,Phòng riêng · 3 phòng tắm · 1 giường · 1 phòng...,"Đà Lạt, Lâm Đồng, Vietnam","Tiện ích gia đình,Đệm bổ sung,Không hút thuốc,...",0,1
2521,Căn hộ chung cư,78.0,Nguyên căn · 2 phòng tắm · 2 giường · 2 phòng ...,"Ba Đình, Hà Nội, Vietnam","Tiện ích gia đình,Phù hợp với trẻ nhỏ,Không hú...",0,1
9018,Khác,25.0,Phòng riêng · 1 phòng tắm · 2 giường · 1 phòng...,"Hoàn Kiếm, Hà Nội, Vietnam","Tiện ích gia đình,Không hút thuốc,Tiện ích bếp...",1,0
11468,Khác,20.0,Nguyên căn · 3 phòng tắm · 12 giường · 1 phòng...,"Tân Bình, Hồ Chí Minh, Vietnam","Tiện ích gia đình,Không hút thuốc,Tiện ích phò...",1,0
12199,Căn hộ dịch vụ,60.0,Phòng riêng · 2 phòng tắm · 1 giường · 1 phòng...,"Quận 3, Hồ Chí Minh, Vietnam","Tiện ích gia đình,Không hút thuốc,Tiện ích bếp...",1,0


In [16]:
train_X_df.dtypes

type           object
area          float64
room_types     object
location       object
utilities      object
inweek          int64
weekend         int64
dtype: object

### Xử lý cột room_types

Ta thấy cột room_types: cho biết thể loại cho thuê và số lượng các phòng mỗi loại

In [17]:
roomtypes=train_X_df['room_types'].str.split(" · ")\
.apply(lambda x: [[x[0]]]+\
       [i.split(" ",1) for i in x[1:-1]]+[x[-1].split(" ",2)[:-1]]+\
       [[re.findall(r'\d+',x[-1].split(" ",2)[-1])[0],"khách tối đa"]])
roomtypes

ID
64       [[Phòng riêng], [3, phòng tắm], [1, giường], [...
2521     [[Nguyên căn], [2, phòng tắm], [2, giường], [2...
9018     [[Phòng riêng], [1, phòng tắm], [2, giường], [...
11468    [[Nguyên căn], [3, phòng tắm], [12, giường], [...
12199    [[Phòng riêng], [2, phòng tắm], [1, giường], [...
                               ...                        
8909     [[Nguyên căn], [1, phòng tắm], [1, giường], [1...
5727     [[Nguyên căn], [1, phòng tắm], [2, giường], [2...
10979    [[Nguyên căn], [1, phòng tắm], [1, giường], [1...
5804     [[Nguyên căn], [2, phòng tắm], [2, giường], [2...
14602    [[Nguyên căn], [2, phòng tắm], [3, giường], [3...
Name: room_types, Length: 8982, dtype: object

Ta thấy các dòng của cột room_types đều cho biết các thông tin: loại phòng, số phòng tắm, số giường, số phòng ngủ, số khách, số khách tối đa. Ta thấy thuộc tính số khách chính là số khách tối thiểu. Do đó khi xử lý cột này ta sẽ sinh ra thêm 6 cột: loại phòng, số phòng tắm, số giường, số phòng ngủ, số khách tối thiểu, số khách tối đa.

In [18]:
class Room_TypesTransform(BaseEstimator, TransformerMixin):    
    def fit(self, X_df, y=None):
        return self
    def transform(self, X_df, y=None):
        # Copy ra 1 dataframe mới để không thay đổi X_df
        X_df_transform=X_df.copy()
        
        # Rút trích các thông tin có trong cột room_types
        roomtypes=X_df_transform['room_types'].str.split(" · ")\
        .apply(lambda x: [[x[0]]]+\
               [i.split(" ",1) for i in x[1:-1]]+[x[-1].split(" ",2)[:-1]]+\
               [[re.findall(r'\d+',x[-1].split(" ",2)[-1])[0],"khách tối đa"]])
        
        # Thêm cột kind of rooms
        X_df_transform['kind of rooms']=roomtypes.apply(lambda x:x[0][0])
        def numeric(x):
            try:
                x = int(x)
            except:
                x=0
            return x
        
        # Thêm cột bathrooms, tiến thành chuyển kiểu số        
        X_df_transform['bathrooms']=roomtypes.apply(lambda x:x[1][0])        
        X_df_transform['bathrooms']= X_df_transform['bathrooms'].apply(numeric)
        
        # Thêm cột bathrooms, loại các dòng không thể ép thành số, tiến thành chuyển kiểu số
        X_df_transform['beds']=roomtypes.apply(lambda x:x[2][0])
        X_df_transform['beds']=X_df_transform['beds'].apply(numeric)
        
        # Thêm cột bathrooms, loại các dòng không thể ép thành số, tiến thành chuyển kiểu số
        X_df_transform['bedrooms']=roomtypes.apply(lambda x:x[3][0])
        X_df_transform['bedrooms']=X_df_transform['bedrooms'].apply(numeric)
        
        # Thêm cột bathrooms, loại các dòng không thể ép thành số, tiến thành chuyển kiểu số
        X_df_transform['min guests']=roomtypes.apply(lambda x:x[4][0])
        X_df_transform['min guests']=X_df_transform['min guests'].apply(numeric)
        
        # Thêm cột bathrooms, loại các dòng không thể ép thành số, tiến thành chuyển kiểu số
        X_df_transform['max guests']=roomtypes.apply(lambda x:x[5][0])
        X_df_transform['max guests']=X_df_transform['max guests'].apply(numeric)
        
        # Bỏ cột room_types  
        X_df_transform=X_df_transform.drop("room_types",axis=1)
        return X_df_transform

In [19]:
room_types_transform=Room_TypesTransform()
train_X_df=room_types_transform.transform(train_X_df)
train_X_df.head()

,type,area,location,utilities,inweek,weekend,kind of rooms,bathrooms,beds,bedrooms,min guests,max guests
ID,,,,,,,,,,,,
64,Nhà riêng,9.0,"Đà Lạt, Lâm Đồng, Vietnam","Tiện ích gia đình,Đệm bổ sung,Không hút thuốc,...",0,1,Phòng riêng,3,1,1,1,2
2521,Căn hộ chung cư,78.0,"Ba Đình, Hà Nội, Vietnam","Tiện ích gia đình,Phù hợp với trẻ nhỏ,Không hú...",0,1,Nguyên căn,2,2,2,2,5
9018,Khác,25.0,"Hoàn Kiếm, Hà Nội, Vietnam","Tiện ích gia đình,Không hút thuốc,Tiện ích bếp...",1,0,Phòng riêng,1,2,1,3,4
11468,Khác,20.0,"Tân Bình, Hồ Chí Minh, Vietnam","Tiện ích gia đình,Không hút thuốc,Tiện ích phò...",1,0,Nguyên căn,3,12,1,12,12
12199,Căn hộ dịch vụ,60.0,"Quận 3, Hồ Chí Minh, Vietnam","Tiện ích gia đình,Không hút thuốc,Tiện ích bếp...",1,0,Phòng riêng,2,1,1,2,3


In [20]:
train_X_df.dtypes

type              object
area             float64
location          object
utilities         object
inweek             int64
weekend            int64
kind of rooms     object
bathrooms          int64
beds               int64
bedrooms           int64
min guests         int64
max guests         int64
dtype: object

### Xử lý cột location

In [21]:
x=train_X_df['location'].apply(lambda x:str(x).replace(", Vietnam","").split(",")[-1].strip()).value_counts()
x

Hồ Chí Minh        3147
Hà Nội           2200
Lâm Đồng           1689
Khánh Hòa           816
Bà Rịa Vũng Tàu     706
Quảng Ninh          356
Hà Nội               67
Vũng Tàu              1
Name: location, dtype: int64

#### Ta thấy Hà Nội xuất hiện 2 lần chắc là do sai khác kiểu lưu trữ, Vũng tàu và bà rịa vũng tàu tương đương nhau nên ta tiến hành convert lại

In [22]:
class ConvertTransform(BaseEstimator, TransformerMixin):    
    def fit(self, X_df, y=None):
        return self
    def transform(self, X_df, y=None):
        # Copy ra 1 dataframe mới để không thay đổi X_df
        X_df_transform=X_df.copy()
        
        def convert(x):
            if x.encode()==b'H\xc3\xa0 N\xe1\xbb\x99i':
                x='Hà Nội'
            elif x.encode()==b'Ha\xcc\x80 N\xc3\xb4\xcc\xa3i':
                x='Hà Nội'
            elif x=="Bà Rịa Vũng Tàu":
                x='Vũng Tàu'
            return x
        
        location_convert=X_df_transform['location']\
        .apply(lambda x:str(x).replace(", Vietnam","").split(",")[-1].strip()).apply(convert)
        
        
        X_df_transform['location']=location_convert
        return X_df_transform

In [23]:
convertTransform=ConvertTransform()

train_X_df=convertTransform.transform(train_X_df)
train_X_df['location'].value_counts()

Hồ Chí Minh    3147
Hà Nội         2267
Lâm Đồng       1689
Khánh Hòa       816
Vũng Tàu        707
Quảng Ninh      356
Name: location, dtype: int64

In [24]:
class LocationTransform(BaseEstimator, TransformerMixin):
    def __init__(self, num_top_locations=1):
        self.num_top_locations= num_top_locations
    def fit(self, X_df, y=None):
        locations_col = X_df['location']
        self.location_counts_ = locations_col.value_counts()
        locations = list(self.location_counts_.index)
        self.top_locations_ = locations[:max(1, min(self.num_top_locations, len(locations)))]
        return self
    def transform(self, X_df, y=None):
        # Copy ra 1 dataframe mới để không thay đổi X_df
        X_df_transform=X_df.copy()
        
        # Lấy cột type
        location_s=X_df_transform['location']
        
        # Chuyển các giá trị trong cột type về các giá trị trong top_types_
        location_s=location_s.apply(lambda x: x if x in self.top_locations_ else "Others")
        
        # Thêm cột title vào dataframe
        X_df_transform['location']=location_s
        
        return X_df_transform

In [25]:
locationTransform=LocationTransform(num_top_locations=3)
locationTransform.fit(train_X_df)
train_X_df=locationTransform.transform(train_X_df)
train_X_df

,type,area,location,utilities,inweek,weekend,kind of rooms,bathrooms,beds,bedrooms,min guests,max guests
ID,,,,,,,,,,,,
64,Nhà riêng,9.0,Lâm Đồng,"Tiện ích gia đình,Đệm bổ sung,Không hút thuốc,...",0,1,Phòng riêng,3,1,1,1,2
2521,Căn hộ chung cư,78.0,Hà Nội,"Tiện ích gia đình,Phù hợp với trẻ nhỏ,Không hú...",0,1,Nguyên căn,2,2,2,2,5
9018,Khác,25.0,Hà Nội,"Tiện ích gia đình,Không hút thuốc,Tiện ích bếp...",1,0,Phòng riêng,1,2,1,3,4
11468,Khác,20.0,Hồ Chí Minh,"Tiện ích gia đình,Không hút thuốc,Tiện ích phò...",1,0,Nguyên căn,3,12,1,12,12
12199,Căn hộ dịch vụ,60.0,Hồ Chí Minh,"Tiện ích gia đình,Không hút thuốc,Tiện ích bếp...",1,0,Phòng riêng,2,1,1,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...
8909,Căn hộ chung cư,75.0,Hà Nội,"Tiện ích bếp,Bếp điện,Tủ lạnh,Tiện ích giải tr...",1,0,Nguyên căn,1,1,1,2,2
5727,Căn hộ dịch vụ,65.0,Hồ Chí Minh,"Tiện ích,Wifi,TV,Điều hoà,Máy giặt,Dầu gội, dầ...",0,1,Nguyên căn,1,2,2,2,4
10979,Khác,28.0,Hồ Chí Minh,"Tiện ích gia đình,Không hút thuốc,Tiện ích bếp...",1,0,Nguyên căn,1,1,1,2,3


### Xử lý cột Ultilies

In [26]:
class UltiliesDropper(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        self.X=X_df
        return self
    def transform(self, X_df, y=None):
        # Copy ra 1 dataframe mới để không thay đổi X_df
        X_df_transform=X_df.copy()
        
        # Bỏ cột Ultilies
        # X_df_transform=X_df_transform.drop("Ultilies",axis=1)
        del X_df_transform['utilities']
        return X_df_transform

In [27]:
ultiliesdropper=UltiliesDropper()
ultiliesdropper.transform(train_X_df)

,type,area,location,inweek,weekend,kind of rooms,bathrooms,beds,bedrooms,min guests,max guests
ID,,,,,,,,,,,
64,Nhà riêng,9.0,Lâm Đồng,0,1,Phòng riêng,3,1,1,1,2
2521,Căn hộ chung cư,78.0,Hà Nội,0,1,Nguyên căn,2,2,2,2,5
9018,Khác,25.0,Hà Nội,1,0,Phòng riêng,1,2,1,3,4
11468,Khác,20.0,Hồ Chí Minh,1,0,Nguyên căn,3,12,1,12,12
12199,Căn hộ dịch vụ,60.0,Hồ Chí Minh,1,0,Phòng riêng,2,1,1,2,3
...,...,...,...,...,...,...,...,...,...,...,...
8909,Căn hộ chung cư,75.0,Hà Nội,1,0,Nguyên căn,1,1,1,2,2
5727,Căn hộ dịch vụ,65.0,Hồ Chí Minh,0,1,Nguyên căn,1,2,2,2,4
10979,Khác,28.0,Hồ Chí Minh,1,0,Nguyên căn,1,1,1,2,3


In [28]:
transform_column_pipeline=make_pipeline(LinkNameDropper(),\
                                        TypeTransform(num_top_types=3),\
                                        AreaTransform(),\
                                        Room_TypesTransform(),\
                                        ConvertTransform(),\
                                        UltiliesDropper(),\
                                        LocationTransform(num_top_locations=3)
                                        )

nume_cols = ['area', 'inweek', 'weekend','bathrooms','beds','bedrooms','min guests','max guests' ]
cate_cols = ['type', 'location', 'kind of rooms']
nume_transform=Pipeline(steps=[('knn', KNNImputer(n_neighbors=3))])

# Tạo 1 pipeline điền giá trị mode và onehotencoder
cate_transform=Pipeline(steps=[('imp_mode',SimpleImputer(missing_values=np.nan, strategy='most_frequent'))
                                ,('encoder',OneHotEncoder(handle_unknown='ignore'))])

# Tạo 1 ColumnTransformer xác định transformer với cột tương ứng
column_transform = ColumnTransformer(transformers=[('num', nume_transform, nume_cols)
                                                   ,('cat', cate_transform, cate_cols)])

preprocess_pipeline=make_pipeline(transform_column_pipeline,column_transform,StandardScaler())

preprocessed_train_X=preprocess_pipeline.fit_transform(X_df)

In [31]:
preprocessed_train_X.shape

(8982, 18)

In [ ]:
preprocessed_val_X=preprocess_pipeline.transform(val_X_df)

In [ ]:
preprocessed_val_X.shape

In [ ]:
# Tính độ đo r^2 trên tập huấn luyện
def compute_mse(y, preds):
    return ((y - preds) ** 2).mean()
def compute_rr(y, preds, baseline_preds):
    return 1 - compute_mse(y, preds) / compute_mse(y, baseline_preds)
baseline_preds = train_y_sr.mean()

In [ ]:
train_X_df=X_df
full_pipeline=Pipeline(steps=[('preprocess',preprocess_pipeline),
                                ('mlpregressor',MLPRegressor(hidden_layer_sizes=(300),early_stopping=True,\
                                                             activation='relu',max_iter=3000,
                                                               solver='lbfgs',random_state=0))])
# full_pipeline.set_params(preprocess__pipeline__typetransform__num_top_types=1,\
#                                      preprocess__pipeline__locationtransform__num_top_locations=2,\
#                                      mlpregressor__alpha=0.01)
# neural_net_model=full_pipeline.fit(train_X_df,train_y_sr)
# compute_rr(val_y_sr, neural_net_model.predict(val_X_df), baseline_preds)

In [ ]:
# Thử nghiệm với các giá trị khác nhau của các siêu tham số
# và chọn ra các giá trị tốt nhất
train_X_df=X_df
train_rrs = []
val_rrs = []
alphas = [0.1, 1, 10, 100, 1000]
num_top_types = [1,2,3,4,5]
num_top_locations=[1,2,3,4,5]
best_val_rr = 0; best_alpha = None; best_num_top_types = None;best_num_top_locations=None
for alpha in alphas:
    for num_top_type in num_top_types:
        for num_top_location in num_top_locations:
            # set params cho mô hình
            full_pipeline.set_params(preprocess__pipeline__typetransform__num_top_types=num_top_type,\
                                     preprocess__pipeline__locationtransform__num_top_locations=num_top_location,\
                                     mlpregressor__alpha=alpha)

            # Fit dữ liệu vào mô hình
            neural_net_model=full_pipeline.fit(train_X_df,train_y_sr)

            # Tính train_err và thêm vào train_errs
            
            train_rrs.append(compute_rr(train_y_sr, neural_net_model.predict(train_X_df), baseline_preds))

            # Tính val_errs và thêm vào val_errs
            val_rrs.append(compute_rr(val_y_sr, neural_net_model.predict(val_X_df), baseline_preds))
            f = open("model.csv", "a",encoding="utf-8")
            f.write(f"{alpha}\t{num_top_type}\t{num_top_location}\t{train_rrs[-1]}\t{val_rrs[-1]}\t300\n")
            f.close()
            # Gán lại các giá trị tốt nhất
            if val_rrs[-1]>best_val_rr:
                best_val_rr=val_rrs[-1]
                best_alpha=alpha
                best_num_top_types=num_top_type
                best_num_top_locations=num_top_location
            
'Finish!'

In [ ]:
# YOUR CODE HERE

# Nối 2 tập huẩn luyện + tập validation
X_df=pd.concat([train_X_df,val_X_df])
y_sr=pd.concat([train_y_sr,val_y_sr])

# Set params tốt nhất
full_pipeline.set_params(preprocess__coladderdropper__num_top_titles=best_num_top_titles,mlpclassifier__alpha=best_alpha)

# Train lại
best_neural_net_model=full_pipeline.fit(X_df,y_sr)